In [2]:
from datetime import datetime
import tensorflow as tf
import importlib
import numpy as np
import pandas as pd
from datetime import datetime
now = str(datetime.now().date())

In [3]:
# import config 
import configs.config
importlib.reload(configs.config)
from configs.config import CFG
from utils import config
config = config.Config.from_json(CFG)

## Create Dataset

In [4]:
# create Train Series and Dataset 
import dataloader.dataloader
importlib.reload(dataloader.dataloader)
from dataloader.dataloader import Train_Series,Dataset
try: 
    del series,ds
except:
    pass
from dataloader.dataloader import Test_Series
try: 
    del test_series
except:
    pass

series = Train_Series(config.data,config.paths)
series.createSeries()
test_series = Test_Series(config.data,config.paths)
test_series.createSeries()
ds = Dataset(series,test_series)
ds.split_dataset()

X_train = ds.X_train
y_train = ds.y_train
X_test = ds.X_test
y_test = ds.y_test

print(X_train.shape)
print(X_test.shape)


(4522, 160, 5)
(45, 10, 5)


In [5]:
print(X_train.shape)

(4522, 160, 5)


In [6]:
print(y_train.shape)

(4522, 160, 1)


In [17]:
# ds.X_train = ds.ds[...,:-1]
# ds.y_train = ds.ds[...,-1]
# ds.X_test = ds.ds_test

# X_train = ds.X_train
# y_train = ds.y_train
# X_test = ds.X_test
# y_test = ds.y_test

# print(X_train.shape)
# print(X_test.shape)
# ds.X_train = ds._fit_transform(ds.X_train)
# ds.X_test = ds._transform(ds.X_test)

In [18]:
ds.mean

array([-9.39468357e+00,  3.91284868e-02, -4.09704756e-01, -7.90602653e-05])

In [19]:
ds.epsilon

1e-15

In [20]:
ds.std

array([3.92790684e+01, 8.70884655e-02, 1.85279738e+00, 1.22471412e-02])

## Save Datset to Files 

In [7]:
import dill
ds_path = "./model/saved_ds/ds.pkl"
with open(ds_path,'wb') as file:
    my_pickle = dill.dump(ds,file)

In [21]:
from datetime import datetime
import pickle
now = str(datetime.now().date())
# Include the epoch in the file name (uses `str.format`)
ds_path = "./model/saved_ds/ds_{now}.obj"

with  open(ds_path,'wb') as file:
    exampleObj = pickle.dump(ds,file)


## Call model and compile

In [25]:
from model.model1D import Unet,Unet_Model
import model.model1D
importlib.reload(model.model1D)
try: 
    del unet
except:
    pass

# def MySCCE_Loss(y_true,y_pred):
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, ignore_class= 0)
#     return loss 


unet = Unet(n_classes=5,n_filters=32)
unet.compile(optimizer='adam',
              #loss = MySCCE_Loss,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'],
              run_eagerly=True)
unet.model((160,4)).summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 160, 4)]     0           []                               
                                                                                                  
 downsampling_block_10 (Downsam  ((None, 80, 32),    3520        ['input_5[0][0]']                
 plingBlock)                     (None, 160, 32))                                                 
                                                                                                  
 downsampling_block_11 (Downsam  ((None, 40, 64),    18560       ['downsampling_block_10[0][0]']  
 plingBlock)                     (None, 80, 64))                                                  
                                                                                            

In [23]:
from model.FCModel import FCModel
import model.FCModel
importlib.reload(model.FCModel)
try: 
    del fc_net
except:
    pass

# def MySCCE_Loss(y_true,y_pred):
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, ignore_class= 0)
#     return loss 


fc_net = FCModel(n_classes=5)
fc_net.compile(optimizer='adam',
              #loss = MySCCE_Loss,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['categorical_accuracy'],
              run_eagerly=True)
fc_net.model((150,2)).summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 150, 2)]          0         
                                                                 
 dense_5 (Dense)             (None, 150, 32)           96        
                                                                 
 dense_6 (Dense)             (None, 150, 64)           2112      
                                                                 
 dropout_12 (Dropout)        (None, 150, 64)           0         
                                                                 
 dense_7 (Dense)             (None, 150, 32)           2080      
                                                                 
 dense_8 (Dense)             (None, 150, 16)           528       
                                                                 
 dropout_13 (Dropout)        (None, 150, 16)           0   

## Train Model and save weights and model to files

In [26]:

#train model
history = unet.fit(X_train,y_train,epochs =30,batch_size = 256)





Epoch 1/30
1/1 [==============================] - 9s 9s/step - loss: 1.4910 - sparse_categorical_accuracy: 0.3751
Epoch 2/30
1/1 [==============================] - 7s 7s/step - loss: 1.1929 - sparse_categorical_accuracy: 0.5188
Epoch 3/30
1/1 [==============================] - 7s 7s/step - loss: 0.8287 - sparse_categorical_accuracy: 0.6673
Epoch 4/30
1/1 [==============================] - 7s 7s/step - loss: 0.7357 - sparse_categorical_accuracy: 0.6952
Epoch 5/30
1/1 [==============================] - 7s 7s/step - loss: 0.6818 - sparse_categorical_accuracy: 0.6823
Epoch 6/30
1/1 [==============================] - 7s 7s/step - loss: 0.6027 - sparse_categorical_accuracy: 0.6951
Epoch 7/30
1/1 [==============================] - 7s 7s/step - loss: 0.5619 - sparse_categorical_accuracy: 0.7006
Epoch 8/30
1/1 [==============================] - 7s 7s/step - loss: 0.5423 - sparse_categorical_accuracy: 0.6871
Epoch 9/30
1/1 [==============================] - 7s 7s/step - loss: 0.5151 - sparse_cat

In [ ]:
log_dir = "./model/saved_models/logs_fc_net_{now}"
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [ ]:
def one_hot(x, depth: int):
  x = np.squeeze(np.int64(x),axis = -1)
  return np.take(np.eye(depth), x, axis=0)
print(one_hot(y_train, 3).shape)
history_FC = fc_net.fit(X_train,one_hot(y_train, 3),epochs=5, batch_size=2048,callbacks=[tensorboard])

(75014, 150, 3)
Epoch 1/5
37/37 [==============================] - 19s 523ms/step - loss: 0.3455 - categorical_accuracy: 0.7653
Epoch 2/5
37/37 [==============================] - 20s 546ms/step - loss: 0.3389 - categorical_accuracy: 0.7697
Epoch 3/5
37/37 [==============================] - 21s 559ms/step - loss: 0.3349 - categorical_accuracy: 0.7726
Epoch 4/5
37/37 [==============================] - 20s 549ms/step - loss: 0.3317 - categorical_accuracy: 0.7749
Epoch 5/5
37/37 [==============================] - 21s 572ms/step - loss: 0.3288 - categorical_accuracy: 0.7767


In [27]:

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "model/weights/cp-{epoch:04d}_{now}.ckpt"
# save weights 
unet.save_weights(checkpoint_path.format(epoch=100,now=now))
#save model
models_path = "model/saved_models/unet_{version}_{now}"
unet.save(models_path.format(version = "v1", now = now))

INFO:tensorflow:Assets written to: model/saved_models/unet_v1_2023-11-29\assets


INFO:tensorflow:Assets written to: model/saved_models/unet_v1_2023-11-29\assets


In [ ]:

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "model/weights/fc_net_cp-{epoch:04d}_{now}.ckpt"
# save weights 
fc_net.save_weights(checkpoint_path.format(epoch=100,now=now))
#save model
models_path = "model/saved_models/fc_net_{version}_{now}"
fc_net.save(models_path.format(version = "v1", now = now))

INFO:tensorflow:Assets written to: model/saved_models/fc_net_v1_2023-11-15\assets


INFO:tensorflow:Assets written to: model/saved_models/fc_net_v1_2023-11-15\assets


In [ ]:
# # Include the epoch in the file name (uses `str.format`)
# checkpoint_path = "model/weights/cp-{epoch:04d}.ckpt"

# # Loads the weights
# unet.load_weights(checkpoint_path.format(epoch =100))

# # Re-evaluate the model
# # loss, acc = unet.evaluate(test_images, test_labels, verbose=2)
# # print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
# def create_events(serie,y_pred :np.array):
#     """create df_events 
#     input:
#     y_pred - score  (predicited vaues )"""
#     def detectChange(last_val,current_val):
#         if last_val == -1 or current_val == -1:
#             return False

#         return last_val !=current_val

#     # create empty dataframe
#     df_events = pd.DataFrame(columns=["series_id","step","event","score"])
#     # unpad y_pred
#     if serie.slice_pads is None:
#         y_pred_unpadded = y_pred
#     else:
#         y_pred_unpadded = serie._unpad(y_pred,serie.slice_pads)
#     # create step seg list
#     #         - events - segmentation mask
#     #         -  score   - predicited vaues for chosen event
#     events = np.argmax(y_pred_unpadded,axis = -1,keepdims=True)
#     scores = np.max(y_pred_unpadded,axis=-1,keepdims=True)      


#     for slices_num in range(events.shape[0]):
#         slice = events[slices_num]
#         score = scores[slices_num]
#         for i in range(events.shape[1]):
#             event_val = slice[i][0]
#             event_score = score[i][0]

#             if i == 0:
#                 # do not detect anything during first step
#                 continue
#             elif not detectChange(slice[i-1],event_val):
#                 continue

#             df_events.loc[len(df_events.index)] = [serie.serie_id,i,event_val,event_score]

#     # decode events 
#     decoded_list = {v: k for k, v in serie.__class__.decode_list.items()}
#     df_events["event"] = df_events["event"].map(decoded_list)  
#     # df_events = serie.decode_events(df_events)
#     # save as serie events 
#     return df_events 

In [ ]:
# def predict(model,serie):
#     x = serie.get_correct_slices()
#     x = x[...,:-1]
#     try:
#         y_pred = model.predict(x)
#     except ValueError:
#         print(x.shape)
#         print(x)
#         print(y_pred)
        
#     # print(y_pred.shape)
#     #print(y_pred)
#     serie_events = create_events(serie, y_pred)   
#     return serie_events

In [47]:

from dataloader.dataloader import Serie
importlib.reload(dataloader.dataloader)
try: 
    del Serie
except:
    pass
from dataloader.dataloader import Serie

In [48]:
def predict(model,serie):
    x = serie.get_correct_slices()
    x = x[...,0:4]
    #print(x)
    #print(x.shape)
    try:
        y_pred = model.predict(x)
    except ValueError:
        print(x.shape)
        print(x)

        
    # print(y_pred.shape)
    #print(y_pred)
    #serie_events , raw_prediction = Serie.create_events(serie,y_pred)   
    serie_events  = Serie.create_events(serie,y_pred)  
    #return serie_events, raw_prediction
    return serie_events


In [33]:
# serie= series.series["04f547b8017d"].get_correct_slices()
# serie[serie[:,:,-1] == 2.0].shape

## Validation with train set

In [34]:
## 
raw_predictions = {}
series_pred = {}
train_pred = None
pred_start = False
for serie_id, serie in series.series.items():
    # print(serie_id)
    #serie_events, raw_predictions[serie_id] = predict(unet,serie)
    serie_events = predict(unet,serie)

    if pred_start == False:
        train_pred = serie_events
        pred_start = True
    else:
        train_pred = pd.concat([train_pred, serie_events])
train_pred

(0, 5760, 4)
[]


UnboundLocalError: local variable 'y_pred' referenced before assignment

In [ ]:
## 
raw_predictions = {}
series_pred = {}
train_pred_fc= None
pred_start = False
for serie_id, serie in series.series.items():
    # print(serie_id)
    #serie_events, raw_predictions[serie_id] = predict(unet,serie)
    serie_events = predict(fc_net,serie)

    if pred_start == False:
        train_pred_fc = serie_events
        pred_start = True
    else:
        train_pred_fc = pd.concat([train_pred_fc, serie_events])
train_pred_fc

22/22 [==============================] - 0s 15ms/step
03d92c9f6f8a 149
03d92c9f6f8a 299
03d92c9f6f8a 449
03d92c9f6f8a 599
03d92c9f6f8a 8549
03d92c9f6f8a 8699
03d92c9f6f8a 8849
03d92c9f6f8a 8999
03d92c9f6f8a 9149
03d92c9f6f8a 9299
03d92c9f6f8a 9449
03d92c9f6f8a 9599
03d92c9f6f8a 9749
03d92c9f6f8a 9899
03d92c9f6f8a 10049
03d92c9f6f8a 10199
03d92c9f6f8a 10349
03d92c9f6f8a 10499
03d92c9f6f8a 10649
03d92c9f6f8a 10799
03d92c9f6f8a 10949
03d92c9f6f8a 11099
03d92c9f6f8a 11249
03d92c9f6f8a 11399
03d92c9f6f8a 11549
03d92c9f6f8a 11699
03d92c9f6f8a 11849
03d92c9f6f8a 11999
03d92c9f6f8a 12149
03d92c9f6f8a 12299
03d92c9f6f8a 12449
03d92c9f6f8a 12599
03d92c9f6f8a 12749
03d92c9f6f8a 12899
03d92c9f6f8a 13049
03d92c9f6f8a 13199
03d92c9f6f8a 13349
03d92c9f6f8a 13499
03d92c9f6f8a 13649
03d92c9f6f8a 13799
03d92c9f6f8a 13949
03d92c9f6f8a 14099
03d92c9f6f8a 14249
03d92c9f6f8a 14399
03d92c9f6f8a 14549
03d92c9f6f8a 14699
03d92c9f6f8a 14849
03d92c9f6f8a 14999
03d92c9f6f8a 15449
03d92c9f6f8a 15599
03d92c9f6f8a 1

,series_id,step,event,score
0,03d92c9f6f8a,141,onset,0.6
1,03d92c9f6f8a,253,onset,0.6
2,03d92c9f6f8a,327,onset,0.5
3,03d92c9f6f8a,557,onset,0.6
4,03d92c9f6f8a,8435,onset,0.6
...,...,...,...,...
932,7fd4284b7ee8,292539,onset,0.6
933,7fd4284b7ee8,294669,onset,0.5
934,7fd4284b7ee8,297485,onset,0.4
935,7fd4284b7ee8,297890,onset,0.6


In [ ]:
serie_events

,series_id,step,event,score
0,7fd4284b7ee8,5751,onset,0.5
1,7fd4284b7ee8,6194,onset,0.6
2,7fd4284b7ee8,6445,onset,0.6
3,7fd4284b7ee8,6521,onset,0.6
4,7fd4284b7ee8,6665,onset,0.6
...,...,...,...,...
932,7fd4284b7ee8,292539,onset,0.6
933,7fd4284b7ee8,294669,onset,0.5
934,7fd4284b7ee8,297485,onset,0.4
935,7fd4284b7ee8,297890,onset,0.6


### Check Test Set

In [42]:
test_series.series["038441c925bb"].get_correct_slices().shape

(1, 5760, 4)

In [49]:
raw_predictions = {}
series_pred = {}
y_pred = None
pred_start = False
for serie_id, serie in test_series.series.items():
    #serie_events, raw_predictions[serie_id] = predict(unet,serie)
    serie_events = predict(unet,serie)
    print(serie_events)
    if pred_start == False:
        y_pred = serie_events
        pred_start = True
    else:
        y_pred = pd.concat([y_pred, serie_events])



1/1 [==============================] - 0s 80ms/step
038441c925bb 149
      series_id  step  event  score
0  038441c925bb    15  sleep    1.0
1/1 [==============================] - 0s 81ms/step
Empty DataFrame
Columns: [series_id, step, event, score]
Index: []
1/1 [==============================] - 0s 79ms/step
0402a003dae9 149
      series_id  step  event  score
0  0402a003dae9   149  awake    1.0


d:\DEV\Projekty\PythonProjects\Kaggle\Detect-sleep-state\dataloader\dataloader.py:273: RuntimeWarning: overflow encountered in exp
  # unpad y_pred
d:\DEV\Projekty\PythonProjects\Kaggle\Detect-sleep-state\dataloader\dataloader.py:274: RuntimeWarning: invalid value encountered in divide
  if serie.slice_pads is None:


In [50]:
y_pred

,series_id,step,event,score
0,038441c925bb,15,sleep,1.0
0,0402a003dae9,149,awake,1.0


In [51]:
Serie.decode_list

{1: 'onset', 2: 'onset', 3: 'wakuep', 4: 'wakeup'}

In [ ]:
# from matplotlib import pyplot as plt
# plt.plot(history.history["accuracy"])

# def accuracy(real,pred):
#     comp = real == pred

#     return np.sum(comp)/(real.shape[0]*real.shape[1])

# arg = np.expand_dims(np.argmax(y_dev_pred,axis = -1),axis = -1)
# print("train accuracy : " + str(accuracy(y_dev,arg)))



In [ ]:
# from matplotlib import pyplot as plt
# plt.plot(history.history["accuracy"])

# def accuracy(real,pred):
#     comp = real == pred
#     return np.sum(comp)/(comp.shape[0]*comp.shape[1])


# y_train_pred = unet.predict(X_train)
# print("train accuracy : " + str(accuracy(y_train,y_train_pred)))


# y_test_pred = unet.predict(X_test)
# print("test accuracy : " +  str(accuracy(y_test,y_test_pred)))

In [ ]:
# from IPython.display import FileLink
# import pandas as pd

# # import os
# # os.chdir(r'kaggle/working')'
# filename = 'train_' +'.csv'
# pd.DataFrame(y_pred).reset_index().to_csv(filename, index=True,index_label = "row_id")

# FileLink(filename)



d:\DEV\Projekty\PythonProjects\Kaggle\Detect-sleep-state\train_.csv

In [ ]:

#train_pred = train_pred.dropna().reset_index(drop = True)
# Submit Train series submission
#train_pred.to_csv('train_submission.csv', index=True,index_label = "row_id")
train_pred.to_csv('train_submission.csv', index=False)

train_submission = pd.read_csv("train_submission.csv")
train_submission

,series_id,step,event,score
0,03d92c9f6f8a,2,wakeup,0.9
1,03d92c9f6f8a,3,onset,0.6
2,03d92c9f6f8a,4,wakeup,0.5
3,03d92c9f6f8a,9,onset,0.5
4,03d92c9f6f8a,12,wakeup,0.5
...,...,...,...,...
333910,7fd4284b7ee8,196713,wakeup,0.6
333911,7fd4284b7ee8,236453,onset,0.5
333912,7fd4284b7ee8,248375,wakeup,0.4
333913,7fd4284b7ee8,250362,onset,0.5


In [ ]:
train_pred_fc.to_csv('train_submission_fc.csv', index=False)

train_submission_fc = pd.read_csv("train_submission_fc.csv")
train_submission_fc

,series_id,step,event,score
0,7fd4284b7ee8,1950,wakeup,0.4
1,7fd4284b7ee8,1989,onset,0.5
2,7fd4284b7ee8,1806,wakeup,0.5
3,7fd4284b7ee8,1848,onset,0.6
4,7fd4284b7ee8,1204,wakeup,0.5
...,...,...,...,...
7247,7fd4284b7ee8,220446,onset,0.5
7248,7fd4284b7ee8,276054,wakeup,0.5
7249,7fd4284b7ee8,278040,onset,0.6
7250,7fd4284b7ee8,67558,wakeup,0.5


In [ ]:
train_events = pd.read_csv("./dataloader/datasets/v1/train-events/train_events.csv").drop(columns = ["night","timestamp"])
train_events["score"] = 1.0
train_events

,series_id,event,step,score
0,038441c925bb,onset,4992.0,1.0
1,038441c925bb,wakeup,10932.0,1.0
2,038441c925bb,onset,20244.0,1.0
3,038441c925bb,wakeup,27492.0,1.0
4,038441c925bb,onset,39996.0,1.0
...,...,...,...,...
14505,fe90110788d2,wakeup,560604.0,1.0
14506,fe90110788d2,onset,574620.0,1.0
14507,fe90110788d2,wakeup,581604.0,1.0
14508,fe90110788d2,onset,NaN,1.0


In [ ]:


"""Event Detection Average Precision

An average precision metric for event detection in time series and
video.

"""

import numpy as np
import pandas as pd
import pandas.api.types
from typing import Dict, List, Tuple


class ParticipantVisibleError(Exception):
    pass


# Set some placeholders for global parameters
series_id_column_name = None
time_column_name = None
event_column_name = None
score_column_name = None
use_scoring_intervals = None


def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        tolerances: Dict[str, List[float]],
        series_id_column_name: str,
        time_column_name: str,
        event_column_name: str,
        score_column_name: str,
        use_scoring_intervals: bool = False,
) -> float:
    """Event Detection Average Precision, an AUCPR metric for event detection in
    time series and video.

    This metric is similar to IOU-threshold average precision metrics commonly
    used in object detection. For events occuring in time series, we replace the
    IOU threshold with a time tolerance.

    Submissions are evaluated on the average precision of detected events,
    averaged over timestamp error tolerance thresholds, averaged over event
    classes.

    Detections are matched to ground-truth events within error tolerances, with
    ambiguities resolved in order of decreasing confidence.

    Detailed Description
    --------------------
    Evaluation proceeds in four steps:

    1. Selection - (optional) Predictions not within a series' scoring
    intervals are dropped.
    2. Assignment - Predicted events are matched with ground-truth events.
    3. Scoring - Each group of predictions is scored against its corresponding
    group of ground-truth events via Average Precision.
    4. Reduction - The multiple AP scores are averaged to produce a single
    overall score.

    Selection

    With each series there may be a defined set of scoring intervals giving the
    intervals of time over which zero or more ground-truth events might be
    annotated in that series. A prediction will be evaluated only if it falls
    within a scoring interval. These scoring intervals can be chosen to improve
    the fairness of evaluation by, for instance, ignoring edge-cases or
    ambiguous events.

    It is recommended that, if used, scoring intervals be provided for training
    data but not test data.

    Assignment

    For each set of predictions and ground-truths within the same `event x
    tolerance x series_id` group, we match each ground-truth to the
    highest-confidence unmatched prediction occurring within the allowed
    tolerance.

    Some ground-truths may not be matched to a prediction and some predictions
    may not be matched to a ground-truth. They will still be accounted for in
    the scoring, however.

    Scoring

    Collecting the events within each `series_id`, we compute an Average
    Precision score for each `event x tolerance` group. The average precision
    score is the area under the (step-wise) precision-recall curve generated by
    decreasing confidence score thresholds over the predictions. In this
    calculation, matched predictions over the threshold are scored as TP and
    unmatched predictions as FP. Unmatched ground-truths are scored as FN.

    Reduction

    The final score is the average of the above AP scores, first averaged over
    tolerance, then over event.

    Parameters
    ----------
    solution : pd.DataFrame, with columns:

        `series_id_column_name` identifier for each time series

        `time_column_name` the time of occurence for each event as a numeric type

        `event_column_name` class label for each event

        The solution contains the time of occurence of one or more types of
        event within one or more time series. The metric expects the solution to
        contain the same event types as those given in `tolerances`.

        When `use_scoring_intervals == True`, you may include `start` and `end`
        events to delimit intervals within which detections will be scored.
        Detected events (from the user submission) outside of these events will
        be ignored.

    submission : pd.DataFrame, with columns as above and in addition:

        `score_column_name` the predicted confidence score for the detected event

    tolerances : Dict[str, List[float]]

        Maps each event class to a list of timestamp tolerances used
        for matching detections to ground-truth events.

    use_scoring_intervals: bool, default False

        Whether to ignore predicted events outside intervals delimited
        by `'start'` and `'end'` events in the solution. When `False`,
        the solution should not include `'start'` and `'end'` events.
        See the examples for illustration.

    Returns
    -------
    event_detection_ap : float
        The mean average precision of the detected events.

    Examples
    --------
    Detecting `'pass'` events in football:
    >>> column_names = {
    ...     'series_id_column_name': 'video_id',
    ...     'time_column_name': 'time',
    ...     'event_column_name': 'event',
    ...     'score_column_name': 'score',
    ... }
    >>> tolerances = {'pass': [1.0]}
    >>> solution = pd.DataFrame({
    ...     'video_id': ['a', 'a'],
    ...     'event': ['pass', 'pass'],
    ...     'time': [0, 15],
    ... })
    >>> submission = pd.DataFrame({
    ...     'video_id': ['a', 'a', 'a'],
    ...     'event': ['pass', 'pass', 'pass'],
    ...     'score': [1.0, 0.5, 1.0],
    ...     'time': [0, 10, 14.5],
    ... })
    >>> score(solution, submission, tolerances, **column_names)
    1.0

    Increasing the confidence score of the false detection above the true
    detections decreases the AP.
    >>> submission.loc[1, 'score'] = 1.5
    >>> score(solution, submission, tolerances, **column_names)
    0.6666666666666666...

    Likewise, decreasing the confidence score of a true detection below the
    false detection also decreases the AP.
    >>> submission.loc[1, 'score'] = 0.5  # reset
    >>> submission.loc[0, 'score'] = 0.0
    >>> score(solution, submission, tolerances, **column_names)
    0.8333333333333333...

    We average AP scores over tolerances. Previously, the detection at 14.5
    would match, but adding smaller tolerances gives AP scores where it does
    not match. This results in both a FN, since the ground-truth wasn't
    detected, and a FP, since the detected event matches no ground-truth.
    >>> tolerances = {'pass': [0.1, 0.2, 1.0]}
    >>> score(solution, submission, tolerances, **column_names)
    0.3888888888888888...

    We also average over time series and over event classes.
    >>> tolerances = {'pass': [0.5, 1.0], 'challenge': [0.25, 0.50]}
    >>> solution = pd.DataFrame({
    ...     'video_id': ['a', 'a', 'b'],
    ...     'event': ['pass', 'challenge', 'pass'],
    ...     'time': [0, 15, 0],  # restart time for new time series b
    ... })
    >>> submission = pd.DataFrame({
    ...     'video_id': ['a', 'a', 'b'],
    ...     'event': ['pass', 'challenge', 'pass'],
    ...     'score': [1.0, 0.5, 1.0],
    ...     'time': [0, 15, 0],
    ... })
    >>> score(solution, submission, tolerances, **column_names)
    1.0

    By adding scoring intervals to the solution, we may choose to ignore
    detections outside of those intervals.
    >>> tolerances = {'pass': [1.0]}
    >>> solution = pd.DataFrame({
    ...     'video_id': ['a', 'a', 'a', 'a'],
    ...     'event': ['start', 'pass', 'pass', 'end'],
    ...     'time': [0, 10, 20, 30],
    ... })
    >>> submission = pd.DataFrame({
    ...     'video_id': ['a', 'a', 'a'],
    ...     'event': ['pass', 'pass', 'pass'],
    ...     'score': [1.0, 1.0, 1.0],
    ...     'time': [10, 20, 40],
    ... })
    >>> score(solution, submission, tolerances, **column_names, use_scoring_intervals=True)
    1.0

    """
    # Validate metric parameters
    assert len(tolerances) > 0, "Events must have defined tolerances."
    assert set(tolerances.keys()) == set(solution[event_column_name]).difference({'start', 'end'}),\
        (f"Solution column {event_column_name} must contain the same events "
         "as defined in tolerances.")
    assert pd.api.types.is_numeric_dtype(solution[time_column_name]),\
        f"Solution column {time_column_name} must be of numeric type."

    # Validate submission format
    for column_name in [
        series_id_column_name,
        time_column_name,
        event_column_name,
        score_column_name,
    ]:
        if column_name not in submission.columns:
            raise ParticipantVisibleError(f"Submission must have column '{target_name}'.")

    if not pd.api.types.is_numeric_dtype(submission[time_column_name]):
        raise ParticipantVisibleError(
            f"Submission column '{time_column_name}' must be of numeric type."
        )
    if not pd.api.types.is_numeric_dtype(submission[score_column_name]):
        raise ParticipantVisibleError(
            f"Submission column '{score_column_name}' must be of numeric type."
        )

    # Set these globally to avoid passing around a bunch of arguments
    globals()['series_id_column_name'] = series_id_column_name
    globals()['time_column_name'] = time_column_name
    globals()['event_column_name'] = event_column_name
    globals()['score_column_name'] = score_column_name
    globals()['use_scoring_intervals'] = use_scoring_intervals

    return event_detection_ap(solution, submission, tolerances)


def filter_detections(
        detections: pd.DataFrame, intervals: pd.DataFrame
) -> pd.DataFrame:
    """Drop detections not inside a scoring interval."""
    detection_time = detections.loc[:, time_column_name].sort_values().to_numpy()
    intervals = intervals.to_numpy()
    is_scored = np.full_like(detection_time, False, dtype=bool)

    i, j = 0, 0
    while i < len(detection_time) and j < len(intervals):
        time = detection_time[i]
        int_ = intervals[j]

        # If the detection is prior in time to the interval, go to the next detection.
        if time < int_.left:
            i += 1
        # If the detection is inside the interval, keep it and go to the next detection.
        elif time in int_:
            is_scored[i] = True
            i += 1
        # If the detection is later in time, go to the next interval.
        else:
            j += 1

    return detections.loc[is_scored].reset_index(drop=True)


def match_detections(
        tolerance: float, ground_truths: pd.DataFrame, detections: pd.DataFrame
) -> pd.DataFrame:
    """Match detections to ground truth events. Arguments are taken from a common event x tolerance x series_id evaluation group."""
    detections_sorted = detections.sort_values(score_column_name, ascending=False).dropna()
    is_matched = np.full_like(detections_sorted[event_column_name], False, dtype=bool)
    gts_matched = set()
    for i, det in enumerate(detections_sorted.itertuples(index=False)):
        best_error = tolerance
        best_gt = None

        for gt in ground_truths.itertuples(index=False):
            error = abs(getattr(det, time_column_name) - getattr(gt, time_column_name))
            if error < best_error and gt not in gts_matched:
                best_gt = gt
                best_error = error

        if best_gt is not None:
            is_matched[i] = True
            gts_matched.add(best_gt)

    detections_sorted['matched'] = is_matched

    return detections_sorted


def precision_recall_curve(
        matches: np.ndarray, scores: np.ndarray, p: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    if len(matches) == 0:
        return [1], [0], []

    # Sort matches by decreasing confidence
    idxs = np.argsort(scores, kind='stable')[::-1]
    scores = scores[idxs]
    matches = matches[idxs]

    distinct_value_indices = np.where(np.diff(scores))[0]
    threshold_idxs = np.r_[distinct_value_indices, matches.size - 1]
    thresholds = scores[threshold_idxs]

    # Matches become TPs and non-matches FPs as confidence threshold decreases
    tps = np.cumsum(matches)[threshold_idxs]
    fps = np.cumsum(~matches)[threshold_idxs]

    precision = tps / (tps + fps)
    precision[np.isnan(precision)] = 0
    recall = tps / p  # total number of ground truths might be different than total number of matches

    # Stop when full recall attained and reverse the outputs so recall is non-increasing.
    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)

    # Final precision is 1 and final recall is 0
    return np.r_[precision[sl], 1], np.r_[recall[sl], 0], thresholds[sl]


def average_precision_score(matches: np.ndarray, scores: np.ndarray, p: int) -> float:
    precision, recall, _ = precision_recall_curve(matches, scores, p)
    # Compute step integral
    return -np.sum(np.diff(recall) * np.array(precision)[:-1])


def event_detection_ap(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        tolerances: Dict[str, List[float]],
) -> float:

    # Ensure solution and submission are sorted properly
    solution = solution.sort_values([series_id_column_name, time_column_name])
    submission = submission.sort_values([series_id_column_name, time_column_name])

    # Extract scoring intervals.
    if use_scoring_intervals:
        intervals = (
            solution
            .query("event in ['start', 'end']")
            .assign(interval=lambda x: x.groupby([series_id_column_name, event_column_name]).cumcount())
            .pivot(
                index='interval',
                columns=[series_id_column_name, event_column_name],
                values=time_column_name,
            )
            .stack(series_id_column_name)
            .swaplevel()
            .sort_index()
            .loc[:, ['start', 'end']]
            .apply(lambda x: pd.Interval(*x, closed='both'), axis=1)
        )

    # Extract ground-truth events.
    ground_truths = (
        solution
        .query("event not in ['start', 'end']")
        .reset_index(drop=True)
    )

    # Map each event class to its prevalence (needed for recall calculation)
    class_counts = ground_truths.value_counts(event_column_name).to_dict()

    # Create table for detections with a column indicating a match to a ground-truth event
    detections = submission.assign(matched = False)

    # Remove detections outside of scoring intervals
    if use_scoring_intervals:
        detections_filtered = []
        for (det_group, dets), (int_group, ints) in zip(
            detections.groupby(series_id_column_name), intervals.groupby(series_id_column_name)
        ):
            assert det_group == int_group
            detections_filtered.append(filter_detections(dets, ints))
        detections_filtered = pd.concat(detections_filtered, ignore_index=True)
    else:
        detections_filtered = detections

    # Create table of event-class x tolerance x series_id values
    aggregation_keys = pd.DataFrame(
        [(ev, tol, vid)
         for ev in tolerances.keys()
         for tol in tolerances[ev]
         for vid in ground_truths[series_id_column_name].unique()],
        columns=[event_column_name, 'tolerance', series_id_column_name],
    )

    # Create match evaluation groups: event-class x tolerance x series_id
    detections_grouped = (
        aggregation_keys
        .merge(detections_filtered, on=[event_column_name, series_id_column_name], how='left')
        .groupby([event_column_name, 'tolerance', series_id_column_name])
    )
    ground_truths_grouped = (
        aggregation_keys
        .merge(ground_truths, on=[event_column_name, series_id_column_name], how='left')
        .groupby([event_column_name, 'tolerance', series_id_column_name])
    )
    # Match detections to ground truth events by evaluation group
    detections_matched = []
    for key in aggregation_keys.itertuples(index=False):
        dets = detections_grouped.get_group(key)
        gts = ground_truths_grouped.get_group(key)
        detections_matched.append(
            match_detections(dets['tolerance'].iloc[0], gts, dets)
        )
    detections_matched = pd.concat(detections_matched)

    # Compute AP per event x tolerance group
    event_classes = ground_truths[event_column_name].unique()
    ap_table = (
        detections_matched
        .query("event in @event_classes")
        .groupby([event_column_name, 'tolerance']).apply(
            lambda group: average_precision_score(
                group['matched'].to_numpy(),
                group[score_column_name].to_numpy(),
                class_counts[group[event_column_name].iat[0]],
            )
        )
    )
    # Average over tolerances, then over event classes
    mean_ap = ap_table.groupby(event_column_name).mean().sum() / len(event_classes)

    return mean_ap



In [ ]:
score(
        train_submission,
        train_events,
        {'onset': [0.4],'wakeup':[0.4]},
        "series_id",
        "step",
        "event",
        "score",
        False,
)

7.771125795571212e-06

In [ ]:
score(
        train_submission_fc,
        train_events,
        {'onset': [0.4],'wakeup':[0.4]},
        "series_id",
        "step",
        "event",
        "score",
        False,
)

1.4515052109037072e-05